In [ ]:
from alpaca.data.timeframe import TimeFrame
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
import datetime

API_KEY = "PK1W6UGQYV6DJYD9HEP7"
SECRET_KEY = "ndH61mpUAGkK7jNKM8fgl1EvjUQKf0yDofyYP0pe"

client = CryptoHistoricalDataClient()
request_params = CryptoBarsRequest(
  symbol_or_symbols=["BTC/USD", 'ETH/USD', 'USDT/USD', 'XRP/USD', 'BNB/USD', 'SOL/USD', 'USDC/USD', 'TRX/USD', 'DOGE/USD', 'ADA/USD'],
  timeframe=TimeFrame.Day,
  start=datetime.datetime(2022, 9, 1),
  end=datetime.datetime(2025, 7, 7)
)

btc_bars = client.get_crypto_bars(request_params)

df = btc_bars.df